**Ameya Karnad <br>ak4251 <br>Assignement 1 - Part 2 <br>Personalisation - Theory and Application **

<br>
**1- Setup**

In [ ]:
#Setup
from scipy.optimize import minimize
from scipy.stats import chisquare
import pandas as pd
import numpy as np
data = pd.read_csv("winequality-red.csv", header = 0 , sep = ';')
train, test = np.split(data.sample(frac=1), [int(.8*len(data))])
X_train = train.loc[:, 'fixed acidity': 'alcohol']
y_train = train.loc[:, 'quality']
X_test = test.loc[:,'fixed acidity': 'alcohol']
y_test = test.loc[:, 'quality']

**2- Regression equations and functions** <br><br>
**Equations**

The equation for linear regression model is
$$f(X) = \beta_0 + \sum_{j=1}^{p} X_j\beta_j $$ 

and the equation for finding the residual sum of squares is 

$$RSS(\beta) = \sum_{i=1}^{N} (y_i - f(x_i))^2$$ 

For the optimal value of $\beta$ , the RSS value should be minimised. We will be minimising this using Scipy’s minimize function 

In [21]:
def rss_compute(B, X, y):
    y_hat = np.matmul(X, B)
    error = 0 
    for i in range(len(y_hat)):
        error = error + (y.iat[i]-y_hat[i])**2
    return error




**3- Optimising the Model**

In [172]:
B0 = np.random.normal(0, 1, X_train.shape[1])
result = minimize(fun=rss_compute, x0=B0, args= (X_train,y_train))


The Regression Variable and their values is as below

In [173]:
X_train.head()

Results = { "Attribute" : list(X_train.columns.values), "Results" : result.x}
Results = pd.DataFrame(Results,columns=['Attribute','Results'])
Results


,Attribute,Results
0,fixed acidity,0.011281
1,volatile acidity,-1.042209
2,citric acid,-0.181255
3,residual sugar,0.015613
4,chlorides,-1.668063
5,free sulfur dioxide,0.004820
6,total sulfur dioxide,-0.003756
7,density,4.272971
8,pH,-0.448337
9,sulphates,0.805282


**Q1. What are the qualitative results from your model? Which features seem to be most
important? Do you think that the magnitude of the features in X may affect the
results (for example, the average total sulfur dioxide across all wines is 46.47, but
the average chlorides is only 0.087)**

Ans. From the model, we get the regression coefficients. By the looks of it, some of the features like 'density' , 'chlorides' and 'volatile acidity'  may look to be more important than the other . But the results of the above analysis give only the regression coefficients. The variables are not based on the same scale and hence these regression value are uncomparable. To Compare the regression values, we need to standardise the variables or use normalisation to bring them to the same scale. 

So yes, The magnitude of the features in X affect the results. Let take the case of total sulfur dioxide and chlorides. Lets compare the change in the value of the result in one percentage change of total sulfur dioxide to one percentage change of chlorides


In [64]:
avg_sulfur = 46.47
avg_chloride = 0.087

per_sulfur_chng = 0.01 * avg_sulfur * Results.loc[Results['Attribute'] == 'total sulfur dioxide']['Results'].iat[0]
per_chloride_chng = 0.01 * avg_chloride * Results.loc[Results['Attribute'] == 'chlorides']['Results'].iat[0]
print("Change in Quality for 1 percent change in total sulfur dioxide = " + str(per_sulfur_chng))
print("Change in Quality for 1 percent change in chlorides = " + str(per_chloride_chng))


Change in Quality for 1 percent change in total sulfur dioxide = -0.0013668728280932438
Change in Quality for 1 percent change in chlorides = -0.001506363071627797


Though the value for chlorides was much higher than that of the total sulfur dioxide,  we can see, the percent changes for both the attributes are quite similar

***Q2. How well does your model fit? You should be able to measure the goodness of fit,
RSS, on both the training data and the test data, but only report the results on the
test data. In Machine Learning we almost always only care about how well the
model fits on data that has not been used to fit the model, because we need to use
the model in the future, not the past. Therefore, we only report performance with
holdout data, or test data. ***

Ans. To test the goodness of fit, we will be using the Chi squared test. the test is given by




$$\chi^2 = \sum_{i=1}^{N}\dfrac{(O_i - E_i)^2}{E_i}$$

where 
$O_i$ is the observed value and $E_i$ is the expected value

Consider the ***Null Hypothesis*** "There is no significant difference between the observed and expected values of quality of wine"

In [196]:
obs_train = np.matmul(X_train, result.x)
obs_test = np.matmul(X_test, result.x)

chi_train = chisquare(obs_train,f_exp=y_train.values)
chi_test_stat = chisquare(obs_test,f_exp=y_test.values)[0]
chi_test_p = chisquare(obs_test,f_exp=y_test.values)[1]


In [197]:
print("The Results of the chi square test on the test data is  : ")
print("Chi Statistic = " + str(chi_test_stat))
print("p value = " + str(chi_test_p))

The Results of the chi square test on the test data is  : 
Chi Statistic = 23.712408718664268
p value = 1.0


The P value for the test is 1. So we can say that the model is a good fit and we can say that the null hypothesis is correct

***Q3. Does the end result or RSS change if you try different initial values of β? What
happens if you change the magnitude of the initial β?***

Ans. Let us use different initial values for β

In [141]:
B1 = np.random.normal(100, 500, X_train.shape[1])
result_new = minimize(fun=rss_compute, x0=B1, args= (X_train,y_train))
Results = { "Attribute" : list(X_train.columns.values), "Results" : result.x}
Results = pd.DataFrame(Results,columns=['Attribute','Results'])
Results


,Attribute,Results
0,fixed acidity,0.023623
1,volatile acidity,-1.180579
2,citric acid,-0.373438
3,residual sugar,0.015319
4,chlorides,-1.731452
5,free sulfur dioxide,0.004234
6,total sulfur dioxide,-0.002941
7,density,4.549933
8,pH,-0.579909
9,sulphates,0.809672


As we can see, the change in the initial value of $\beta$ magnitude did not change the results of the RSS function. But it was noticed that the time taken by the execution for the previous value of $\beta$ = (0,1) is less that the current execution  $\beta$ = (100, 500). 

Th result is same in both the cases.

But the RSS does change when there is change in the initial value of B. Lets check the results of change in $\beta$, and try to compare the RSS value for $\beta_0$, $\beta_1$ for the training set 

In [145]:
print("RSS value when B is in the range of 0 and 1 = " + str(rss_compute(B0, X_train, y_train)))

RSS value when B is in the range of 0 and 1 = 3148488.9209281285


In [146]:
print("RSS value when B is in the range of 100 and 500 = " + str(rss_compute(B1, X_train, y_train)))

RSS value when B is in the range of 100 and 500 = 3847022509454.332


***Q4. Does the choice of solver method change the end result or RSS?***

The Choice of the solver method does not affect the RSS value but it may affect the end result. 

Let us check out the final result based on the solver method. We will be using the $\beta_0$ for this analysis with 3 Solvers, Nelder-Mead, Powell and TNC



In [152]:
result_2 = minimize(fun=rss_compute, x0=B0, args= (X_train,y_train),method='Nelder-Mead')
result_2.x

array([-0.0952816 ,  0.47666263,  2.2820462 , -0.044489  , -0.0057541 ,
        0.00715081, -0.00588756,  1.51189172,  0.38619049,  0.70043927,
        0.24918433])

In [153]:
result_3 = minimize(fun=rss_compute, x0=B0, args= (X_train,y_train),method='Powell')
result_3.x

array([ 0.08621255, -1.14076092, -0.0998666 ,  0.01586066, -0.89771127,
        0.00270494, -0.00284484, -0.64799634,  0.92613803,  0.8355615 ,
        0.26000877])

In [155]:
result_4 = minimize(fun=rss_compute, x0=B0, args= (X_train,y_train),method='TNC')
result_4.x

array([ 0.09135315, -0.15715018, -0.84934245, -0.01874912, -1.47197995,
        0.0095186 , -0.00511152,  0.09996068,  0.17095504,  1.01926744,
        0.40173662])

As we can see, the different solver method give out different coefficients. So the solver methods affect the end results.

**4 - Regularisation**



In [231]:
def regularised_rss_compute(B, X, y, C_lambda):
    y_hat = np.matmul(X, B)
    y_reg = y_hat + C_lambda *  np.sum(B**2)
    error = 0 
    for i in range(len(y_reg)):
        error = error + (y.iat[i]-y_reg[i])**2
    return error

In [230]:
lambda_1 = 0.01


In [232]:
result_reg = minimize(fun=regularised_rss_compute, x0=B0, args= (X_train,y_train,lambda_1))
result_reg.x

array([ 1.14671888e-02, -1.04204438e+00, -1.81221356e-01,  1.57036318e-02,
       -1.66786666e+00,  4.81778448e-03, -3.75525809e-03,  4.05654978e+00,
       -4.47477307e-01,  8.05549099e-01,  2.92143703e-01])

In [233]:
X_train.head()

Results_reg = { "Attribute" : list(X_train.columns.values), "Results" : result_reg.x}
Results_reg = pd.DataFrame(Results,columns=['Attribute','Results'])
Results_reg

,Attribute,Results
0,fixed acidity,0.011281
1,volatile acidity,-1.042209
2,citric acid,-0.181255
3,residual sugar,0.015613
4,chlorides,-1.668063
5,free sulfur dioxide,0.004820
6,total sulfur dioxide,-0.003756
7,density,4.272971
8,pH,-0.448337
9,sulphates,0.805282


Above is the Result for the regularised y and for lambda = 0.01
Now lets validate this result with the test data. 

In [234]:
obs_reg_test = np.matmul(X_test, result_reg.x)
chi_reg_test_stat = chisquare(obs_reg_test,f_exp=y_test.values)[0]
chi_reg_test_p = chisquare(obs_reg_test,f_exp=y_test.values)[1]

print("The Results of the chi square test on the test data with regularisation is  : ")
print("Chi Statistic = " + str(chi_reg_test_stat))
print("p value = " + str(chi_reg_test_p))

The Results of the chi square test on the test data with regularisation is  : 
Chi Statistic = 23.728088316993123
p value = 1.0


**Q1. How does RSS on the training data change? How does RSS on the test data change? What happens if you try different values of lambda? Can you roughly tune lambda to get the best results on the test data? **


In [235]:
print("RSS without regularisation for training data is " + str(rss_compute(B0, X_train, y_train)))
print("RSS with regularisation for training data is " + str(regularised_rss_compute(B0, X_train, y_train, C_lambda)))

print("RSS without regularisation for testing data is " + str(rss_compute(B0, X_test, y_test)))
print("RSS with regularisation for testing data is " + str(regularised_rss_compute(B0, X_test, y_test, C_lambda)))

RSS without regularisation for training data is 3148488.9209281285
RSS with regularisation for training data is 3154740.538144643
RSS without regularisation for testing data is 960707.0916872026
RSS with regularisation for testing data is 962429.5493680306


We can see that after regularisation, the RSS value has changed. 

**Lasso Regularisation**

In [236]:
def l1_regularisation_rss_compute(B, X, y, C_lambda):
    y_hat = np.matmul(X, B)
    y_reg = y_hat + C_lambda *  np.sum(abs(B))
    error = 0 
    for i in range(len(y_reg)):
        error = error + (y.iat[i]-y_reg[i])**2
    return error

In [237]:
result_l1_reg = minimize(fun=l1_regularisation_rss_compute, x0=B0, args= (X_train,y_train,lambda_1))
result_l1_reg.x

array([ 1.13602643e-02, -1.04214639e+00, -1.81251290e-01,  1.56501246e-02,
       -1.66793773e+00,  4.81886591e-03, -3.75545087e-03,  4.18480434e+00,
       -4.47948886e-01,  8.05392909e-01,  2.92263589e-01])

In [238]:
Results_l1_reg = { "Attribute" : list(X_train.columns.values), "Results" : result_l1_reg.x}
Results_l1_reg = pd.DataFrame(Results,columns=['Attribute','Results'])
Results_l1_reg

,Attribute,Results
0,fixed acidity,0.011281
1,volatile acidity,-1.042209
2,citric acid,-0.181255
3,residual sugar,0.015613
4,chlorides,-1.668063
5,free sulfur dioxide,0.004820
6,total sulfur dioxide,-0.003756
7,density,4.272971
8,pH,-0.448337
9,sulphates,0.805282


The end results of the L1 regularisation seem to be similar to that of the L2 regularisation. Running the Chi test on the data, gives pvalue as 1 as well. But the chi Statistic value has decrease with respect to thte L2 regularisation 


In [239]:
obs_l1reg_test = np.matmul(X_test, result_l1_reg.x)
chi_l1reg_test_stat = chisquare(obs_l1reg_test,f_exp=y_test.values)[0]
chi_l1reg_test_p = chisquare(obs_l1reg_test,f_exp=y_test.values)[1]

print("The Results of the chi square test on the test data with regularisation is  : ")
print("Chi Statistic = " + str(chi_l1reg_test_stat))
print("p value = " + str(chi_l1reg_test_p))

The Results of the chi square test on the test data with regularisation is  : 
Chi Statistic = 23.07928486762725
p value = 1.0


In [240]:
avg_sulfur = 46.47
avg_chloride = 0.087

per_sulfur_chng = 0.01 * avg_sulfur * Results_l1_reg.loc[Results_l1_reg['Attribute'] == 'total sulfur dioxide']['Results'].iat[0]
per_chloride_chng = 0.01 * avg_chloride * Results_l1_reg.loc[Results_l1_reg['Attribute'] == 'chlorides']['Results'].iat[0]
print("Change in Quality for 1 percent change in total sulfur dioxide = " + str(per_sulfur_chng))
print("Change in Quality for 1 percent change in chlorides = " + str(per_chloride_chng))


Change in Quality for 1 percent change in total sulfur dioxide = -0.0017452609192854633
Change in Quality for 1 percent change in chlorides = -0.0014512151279972394


Looking at the results for the chloride and sulfur change, i still feel that the magnitude of the features in X affect the end value. To get a perfect analysis on whether a feature is really important or not, they have to be bought to the same scale using normalisation techniques. 